# Advent of Code 2024, Day 6

In [ ]:
import numpy as np
from tqdm.auto import tqdm

In [ ]:
def read_input(path: str):
    with open(path) as f:
        code = {".": 0, "#": 1, "^": 2}
        inp = np.array([[code[ch] for ch in ln.strip()] for ln in f])
    start = np.hstack(np.nonzero(inp == 2))
    inp[*start] = 0
    return inp, start

## Part 1

In [ ]:
def walk(grid, start):
    pass


def step(grid, start, dir):
    pass

In [ ]:
grid, start = read_input("06-example.txt")
walk(grid, start)

In [ ]:
s = start.copy().tolist()
d = "u"
p = [s]

while True:
    # Find as far as we can go; if no blocker we're done
    try:
        if d == "u":
            try:
                newr = int(np.argwhere(grid[: s[0], s[1]] == 1).max() + 1)
            except:
                newr = grid.shape[0]
            p += [[r, s[1]] for r in range(s[0] + 1, newr - 1, -1)]
            print("up", s, "->", [newr, s[1]], len(p), p)
            s = [newr, s[1]]
            d = "r"
            continue

        if d == "r":
            newc = int(np.argwhere(grid[s[0], s[1] + 1 :] == 1).min() - 1)
            p += [[s[0], s[1] + 1 + c] for c in range(0, newc + 1, 1)]
            print("rg", s, "->", [s[0], s[1] + 1 + newc], len(p), p)
            s = [s[0], s[1] + 1 + newc]
            d = "d"
            continue

        if d == "d":
            newr = int(np.argwhere(grid[s[0] + 1 :, s[1]] == 1).min() - 1)
            p += [[r, s[1]] for r in range(s[0] + 1, s[0] + 1 + newr + 1, 1)]
            print("dn", s, "->", [s[0] + 1 + newr, s[1]], len(p), p)
            s = [s[0] + 1 + newr, s[1]]
            d = "l"
            continue

        if d == "l":
            newc = int(np.argwhere(grid[s[0], : s[1]] == 1).max() + 1)
            p += [[s[0], c] for c in range(s[1] - 1, newc - 1, -1)]
            print("lf", s, "->", [s[0], newc], len(p), p)
            s = [s[0], newc]
            d = "u"
            continue

    except ValueError:
        print(len(set(tuple(x) for x in p)))
    break

In [ ]:
p, s

In [ ]:
def walk(g, s):
    s = s.copy().tolist()
    print(f"{s = }")

    pos = []

    dir = "up"
    while True:
        # try:
        if dir == "up":
            nr = int(np.argwhere(g[: s[0], s[1]] == 1).max()) - 1
            s1 = [nr, s[1]]
            newpos = [[r, s[1]] for r in range(s[0], nr - 1, -1)]
            print(f"{dir = }: {s = } -> {s1 = }: {newpos}")
            pos += newpos
            s = s1
            dir = "rg"
            continue

        if dir == "rg":
            nc = int(np.argwhere(g[s[0], (s[1] + 1) :] == 1).max())
            s1 = [s[0], s[1] + nc]
            print(f"{dir = }: {s = } -> {s1 = }: {newpos}")
            newpos = [[s[0], c] for c in range(s[1], nc + 1, 1)]
            pos += newpos
            dir = "rg"
            continue

        if dir == "down":
            newy = np.argwhere(g[s[0] :, s[1]] == 1).min()
            pos += [[r, s[1]] for r in np.arange(s[1], newy + 1)]
            s[0] = np.argwhere(g[s[0] :, s[1]] == 1).min()
            dir = "left"
            continue

        if dir == "left":
            s[1] = np.argwhere(g[s[0], : s[1]] == 1).max()
            dir = "up"
            continue
    # except:
    #    return pos


walk(grid, start)

In [ ]:
def step(arr, s, ds=np.array([-1, 0])):
    R0 = np.array([[0, 1], [-1, 0]])
    for d in range(4):
        ds1 = np.linalg.matrix_power(R0, d) @ ds
        s1 = s + ds1

        # If at boundary, return with zero ds
        if (s1 < 0).any() or (s1 >= arr.shape).any():
            ds1[:] = 0
            return s1, ds1

        # If OK, update
        if (arr[*s1] == 0).all():
            return s1, ds1

    raise RuntimeError


def walk(arr, start, ds=np.array([-1, 0])):
    i = 0
    stops = [start]
    
    while np.abs(ds).any() > 0:
        start, ds = step(arr, start, ds)
        
        i += 1
        stops += [start]

        # Check for recursion
        if i >= 4:
            for j in range(0, i-1):
                if (stops[j] == stops[-2]).all() and (stops[j + 1] == stops[-1]).all():
                    raise RecursionError(j, stops[j], j+1, stops[j+1], -2, stops[-2])
    return start, ds, stops


end, dsend, stops = walk(grid, start)
len(set(tuple(x.tolist()) for x in stops)) - 1

### Part 2

An obstruction guarantees a loop.
Too big of a problem to brute-force it via walking.

In [ ]:
grid, start = read_input("06.txt")

In [ ]:
ones = np.stack(np.nonzero(grid==1)).T

In [ ]:
for pt in ones:
    #if not (pt ==[6,1]).all():
    #    continue
    print(f"{pt = }")
    
    eset = np.arange(1, grid.shape[0] - pt[0], 1)
    dset = np.arange(1, grid.shape[1] - pt[1], 1)
    print(f"{eset = }")
    print(f"{dset = }")

    dval = {d for d in dset if ((pt + [1, d]) == ones).all(1).sum() > 0}
    eval = {e for e in eset if ((pt + [e, -1]) == ones).all(1).sum() > 0}
    jval = {(e,d) for d in dset for e in eset if ((pt + [e+1, d-1]) == ones).all(1).sum() > 0 }

    print(f"{dval = }")
    print(f"{eval = }")
    print(f"{jval = }")
    print()

    print({j for j in jval if j[0] in eval or j[1] in dval})

    print()

In [ ]:
grid

In [ ]:
for dr in er:
    for dc in ec:
        test1 = (ones[0] + [1, dc] == ones).all(axis=1).sum() > 0
        if test1:
            print("1", ones[0], 1, dc, ones[0] + [1, dc], f"d={dc}")
            continue
        test2 = (ones[0] + [dr, dc - 1] == ones).all(axis=1).sum() > 0
        if test2:
            print("2", ones[0], dr, dc-1, ones[0] + [dr, dc-1], f"e={dr}", f"d={dc}")
            continue
        test3 = (ones[0] + [dr - 1, -1] == ones).all(axis=1).sum() > 0
        if test3:
            print("3", ones[0], dr, dc, ones[0] + [dr, dc], f"e={dr}")
            continue

In [ ]:
[ for dr in er for dc in ec]

In [ ]:
ones

In [ ]:
for dr in er:
    for dc in ec:
        print(ones[0]+[dr,dc],np.count_nonzero((ones[0]+[dr,dc])== ones,axis=0))

In [ ]:
for i, (r,c) in enumerate(ones):

    break

In [ ]:
ec

In [ ]:
dr

In [ ]:
for dr in er:
    for dc in ec:
        pass

In [ ]:
len(set(tuple(x.tolist()) for x in stops[1:-1]))

In [ ]:
cycles = []
for ir, ic in tqdm(set(tuple(x.tolist()) for x in stops[1:-1])):
    if (start == (ir, ic)).all() or grid[(ir, ic)] == 1:
        continue
    tmpgrid = grid.copy()
    tmpgrid[ir, ic] = 1
    try:
        walk(tmpgrid, start)
    except RecursionError:
        cycles += [[ir,ic]]

len(cycles)